In [1]:
import numpy as np
import pandas as pd
from gates import Network
from sklearn.preprocessing import OneHotEncoder

from optimizers import GeneticOptimizer
from optimizers.genetic import RouletteWheelSelection, RankSelection, NRandomChangesMutation, MultiPointCrossing, HighestAccuracyFitness, LowestMseFitness

## Preparing data

In [2]:
data = pd.read_csv('data/zoo.csv')

In [3]:
legs = data['legs']
other_features = data.drop(columns=['animal_name', 'legs', 'class_type']).to_numpy()
legs_onehot = OneHotEncoder().fit_transform(legs.to_numpy().reshape(-1, 1)).todense()
features = np.hstack([other_features, legs_onehot])
features = np.array(features.astype(bool))

In [4]:
y = data['class_type'].to_numpy()

is_mammal = y == 1
is_bird = y == 2
is_reptile = y == 3
is_fish = y == 4
is_amphibian = y == 5
is_bug = y == 6

## Optimizing network

In [5]:
network = Network(21, [40, 50, 50, 40])
network.evaluate(features, is_fish)

In [7]:
optimizer = GeneticOptimizer(
    fitness=LowestMseFitness(),
    crossing=MultiPointCrossing(3),
    mutation=NRandomChangesMutation(4),
    selection=RouletteWheelSelection(),
    cross_prob=0.7,
    mutate_prob=0.2,
    n_chromosomes=100,
    n_generations=200,
    n_elite=4,
    patience=30,
)

optimizer.optimize(network, features, is_fish)

Optimizing 905 parameters


Evolution:   3%|▎         | 6/200 [00:05<02:41,  1.20it/s, fitness: inf, accuracy: 0.960]               


In [9]:
network.evaluate(features, is_fish)

1.0